In [1]:
from src.factor.optimizer.api import PortfolioOptimizer
config_path = 'custom_opt_config.yaml'

optim = PortfolioOptimizer('quadprog')
optim.setup_optimizer(config_path)

s0 = optim.optimize(20240530)
s0


now using random u!!!!!!


PortOptimResult ---------- 
Information : 
    is_success = True ,
    status     = optimal ,
Result : 
    utility    = Utility(Total Utility=[2.2409],alpha=[2.5093],square=[-0.2684],short=[-0.0000])
    accuracy   = Accuarcy(Is Accurate=[True] ,,
          lin_ub_bias=(√)6.122275686415435e-09,
          lin_lb_bias=(√)-6.122275797437737e-09,
          bnd_ub_bias=(√)0.0,
          bnd_lb_bias=(√)-6.260159294157552e-09
Portfolio <date=20240530> <name=port> <value=None> : 
      secid    weight
0    300760  0.032973
1    601169  0.031407
2    300655  0.030826
3    600015  0.030720
4    600958  0.030696
..      ...       ...
99   601869  0.000005
100  600558  0.000002
101  601717  0.000001
102     596  0.000001
103  300257  0.000001

[104 rows x 2 columns]
Analytic : 
 portfolio benchmark active
style   
size -0.7714 -0.5875 -0.1839
beta 0.5912 0.5932 -0.0020
momentum -0.2794 -0.2591 -0.0203
residual_volatility -0.2654 -0.2621 -0.0032
non_linear_size 0.4849 0.5821 -0.0971
book_to_price 

In [2]:
s0.port = s0.port.forward(20240606)
s1 = optim.optimize(20240606 , initial_port=s0.port)
s1

now using random u!!!!!!


PortOptimResult ------> 
Information : 
    is_success = True ,
    status     = optimal ,
Result : 
    utility    = Utility(Total Utility=[0.2393],alpha=[0.4550],square=[-0.2155],turnover=[-0.0002],short=[-0.0000])
    accuracy   = Accuarcy(Is Accurate=[True] ,,
          lin_ub_bias=(√)1.0981570097712279e-09,
          lin_lb_bias=(√)-1.7565824350640469e-09,
          bnd_ub_bias=(√)-1.2800777451915657e-10,
          bnd_lb_bias=(√)-5.622228448614264e-10,
          excess_turn=(X)-1.5161760266724222e-06
Portfolio <date=20240606> <name=port> <value=None> : 
      secid    weight
0    601658  0.030912
1      2004  0.030882
2      2060  0.025924
3    605333  0.024511
4      2142  0.022980
..      ...       ...
115  688271  0.000004
116  600882  0.000003
117    2897  0.000002
118     938  0.000001
119  301207  0.000001

[120 rows x 2 columns]
Other components include ['w' , 'secid'])

In [3]:
from src.factor.basic import RISK_MODEL
risk_model = RISK_MODEL.get(20240606)

In [4]:
s0.port

Portfolio <date=20240606> <name=port> <value=None> : 
      secid    weight
0    605111  0.032345
1    601658  0.031736
2      2004  0.030961
3      2060  0.025924
4    605333  0.024511
..      ...       ...
101  688271  0.000004
102  600882  0.000003
103    2897  0.000002
104     938  0.000001
105  301207  0.000001

[106 rows x 2 columns]

In [74]:
from argparse import Namespace
from dataclasses import dataclass
import pandas as pd
import numpy as np
from src.environ import RISK_INDUS , RISK_STYLE

from typing import Any , ClassVar , Literal , Optional

@dataclass
class RiskProfile:
    factors  : Optional[np.ndarray | list] = None
    exposure : Optional[np.ndarray] = None
    variance : Optional[float] = None

    variance_measure : ClassVar[list[str]] = ['variance','std_dev']

    def __bool__(self): return self.factors is not None
    @property
    def standard_deviation(self): return np.sqrt(self.variance) if self.variance is not None else None
    def to_dataframe(self):
        if not self: return None
        df0 = pd.DataFrame({'factor_name':self.factors,'value':self.exposure})
        df1 = pd.DataFrame({'factor_name':self.variance_measure,'value':[self.variance,self.standard_deviation]})
        return pd.concat([df0,df1]).set_index('factor_name')
    
@dataclass
class Analytic:
    industry : pd.DataFrame | Any = None
    style    : pd.DataFrame | Any = None
    risk     : pd.DataFrame | Any = None

    def __post_init__(self):
        ...

    def __bool__(self): return self.industry is not None

    def append(self , port_type : Literal['portfolio' , 'benchmark' , 'initial' , 'active'] , risk_profile : RiskProfile):
        df = risk_profile.to_dataframe()
        if df is None: return
        industry = df.loc[RISK_INDUS].rename(columns={'value':port_type}).rename_axis('industry',axis='index')
        style    = df.loc[RISK_STYLE].rename(columns={'value':port_type}).rename_axis('style',axis='index')
        risk     = df.loc[RiskProfile.variance_measure].rename(columns={'value':port_type}).rename_axis('measure',axis='index')
        
        if self:
            self.industry = pd.concat([self.industry , industry] , axis = 1)
            self.style    = pd.concat([self.style    , style   ] , axis = 1)
            self.risk     = pd.concat([self.risk     , risk    ] , axis = 1)
        else:
            self.industry = industry
            self.style    = style
            self.risk     = risk

    def styler(self):
        return type(self)(
            self.industry.style.format(lambda x:f'{x:.2%}') ,
            self.style.style.format(lambda x:f'{x:.4f}') ,
            self.risk.style.format(lambda x:f'{x:.4%}')
        )
        

def common_exposure(self , port):
    rmodel_f = self.F.loc[port.secid , self.common_factors]
    return port.weight.dot(rmodel_f)

def specific_risk(self , port):
    rmodel_s = self.S.loc[port.secid].values.flatten()
    return (port.weight * rmodel_s).dot(port.weight)

def analysis(self , port = None):
    if port is None: return RiskProfile()
    common_exp = common_exposure(self  , port)
    variance = common_exp.dot(self.C).dot(common_exp.T) + specific_risk(self , port)
    return RiskProfile(self.common_factors , common_exp , variance)

def analyze(self , port , bench = None , init = None):
    rslt = Analytic()
    rslt.append('portfolio' , analysis(self , port))
    rslt.append('initial'   , analysis(self , init))
    rslt.append('benchmark' , analysis(self , bench))
    rslt.append('active'    , analysis(self , None if bench is None else port - bench))
    return rslt

analysis(risk_model , s0.port) , analysis(risk_model , s1.port)

(RiskProfile(factors=['market', 'petro', 'coal', 'nonferrous', 'utility', 'public', 'steel', 'chemical', 'construct', 'cement', 'material', 'light', 'machine', 'power', 'defense', 'auto', 'retail', 'leisure', 'appliance', 'textile', 'health', 'liqor', 'food', 'agro', 'bank', 'financial', 'estate', 'transport', 'marine', 'airline', 'electronic', 'telecom', 'hardware', 'software', 'media', 'complex', 'size', 'beta', 'momentum', 'residual_volatility', 'non_linear_size', 'book_to_price', 'liquidity', 'earnings_yield', 'growth', 'leverage'], exposure=array([ 1.        ,  0.01992276,  0.0097974 ,  0.04099828,  0.06266961,
         0.01035758,  0.        ,  0.04561095,  0.01090874,  0.01214167,
         0.0153558 ,  0.01908723,  0.06660478,  0.06523091,  0.03647615,
         0.04151588,  0.01933528,  0.        ,  0.01816021,  0.00366443,
         0.10107917,  0.02547328,  0.00912022,  0.00815236,  0.05584448,
         0.03245167,  0.01721723,  0.01977669,  0.00127848,  0.01389815,
         0.

In [79]:
import pandas as pd
def analyze(self , port , bench = None , init = None):
    rslt = Analytic()
    rslt.append('portfolio' , analysis(self , port))
    rslt.append('initial'   , analysis(self , init))
    rslt.append('benchmark' , analysis(self , bench))
    rslt.append('active'    , analysis(self , None if bench is None else port - bench))
    return rslt
rslt = analyze(risk_model , s0.port , optim.opt_input.benchmark_port , s1.port)
rslt.styler().risk

,portfolio,initial,benchmark,active
measure,,,,
variance,4.8792%,4.8208%,4.3142%,0.4647%
std_dev,22.0890%,21.9563%,20.7707%,6.8167%


In [63]:
rslt

Analytic(industry=              active
industry            
petro       0.010528
coal       -0.009590
nonferrous -0.009459
utility     0.031016
public     -0.002657
steel      -0.008701
chemical   -0.007626
construct  -0.009722
cement      0.008758
material    0.009958
light       0.009442
machine     0.014171
power      -0.010412
defense     0.007098
auto        0.000283
retail      0.008589
leisure    -0.002738
appliance  -0.001147
textile    -0.003524
health     -0.013801
liqor      -0.007463
food       -0.009524
agro       -0.010732
bank        0.008790
financial  -0.008814
estate      0.000274
transport   0.005482
marine     -0.010426
airline     0.010526
electronic  0.005088
telecom    -0.000739
hardware   -0.007702
software   -0.009814
media       0.015680
complex    -0.001092, style=                       active
style                        
size                -0.230549
beta                 0.000095
momentum            -0.076576
residual_volatility -0.057566
non_linear_size   

In [ ]:
d

In [47]:
analysis(risk_model)

RiskProfile(factors=None, exposure=None, variance=None)

In [5]:
s.port.port.reset_index(drop=True)

,secid,weight
0,601006,0.025522
1,600657,0.018860
2,688100,0.018175
3,778,0.017457
4,601108,0.017248
...,...,...
312,2679,-0.008031
313,688565,-0.008247
314,300955,-0.008937
315,300329,-0.011669


TypeError: DataFrame.merge() got an unexpected keyword argument 'by'

In [12]:
optim.solver.input.cov_con.S

array([1.9106474, 2.312497 , 4.0749316, ..., 3.564367 , 3.55709  ,
       6.9876666], dtype=float32)